In [1]:
%load_ext Cython

In [42]:
%%cython -c=-std=c++17
# distutils: language = c++

# Importing the required C++ types for the Afhel API: vector, string & bool
from libcpp.vector cimport vector
from libcpp.string cimport string
from libc.stdint cimport int64_t
from libcpp cimport bool


cdef extern from "<iostream>" namespace "std":
    cdef cppclass ostream:
        ostream& write(const char*, int) except +
    cdef cppclass istream:
        istream& read(const char*, int) except +


cdef extern from "<fstream>" namespace "std":
    cdef cppclass ofstream(ostream):
        # constructors
        ofstream(const char*) except +
    cdef cppclass ifstream(ostream):
        # constructors
        ifstream(const char*) except +
        
        
        
cdef extern from "seal/plaintext.h" namespace "seal":
    cdef cppclass Plaintext:
        Plaintext() except +
        Plaintext(const Plaintext &copy) except +
        Plaintext(Plaintext &&source) except +
        bool is_zero() except +
        void save(ostream &stream) except +
        void load(istream &stream) except +
        
        
        
cdef extern from "seal/ciphertext.h" namespace "seal":
    cdef cppclass Ciphertext:
        Ciphertext() except +
        Ciphertext(const Ciphertext &copy) except +
        Ciphertext(Ciphertext &&source) except +
        int size_capacity() except +
        int size() except +
        void save(ostream &stream) except +
        void load(istream &stream) except +
        


EXAMPLE ON USAGE OF STD STREAMS FROM C++

In [54]:
%%cython
# distutils: language = c++

cdef extern from "<iostream>" namespace "std":
    cdef cppclass ostream:
        ostream& write(const char*, int) except +

# obviously std::ios_base isn't a namespace, but this lets
# Cython generate the correct C++ code
cdef extern from "<iostream>" namespace "std::ios_base":
    cdef cppclass open_mode:
        pass
    cdef open_mode binary
    # you can define other constants as needed

cdef extern from "<fstream>" namespace "std":
    cdef cppclass ofstream(ostream):
        # constructors
        ofstream(const char*) except +

def test_ofstreamss(bytes s):
    cdef ofstream* outputter
    # use try ... finally to ensure destructor is called
    outputter = new ofstream("output.txt")
    try:
        outputter.write(s,len(s))
    finally:
        del outputter

In [56]:
test_ofstreamss(b"we write more aaaastuff")